In [2]:
from multi_arm import MultiArmedBanditEnv

env: MultiArmedBanditEnv = MultiArmedBanditEnv(n_arms=5, seed=42)
obs: int = env.reset()
done: bool = False
total_reward: int = 0

print('Multi-armed state', env.probabilities)
while not done:
    action: int = env.action_space.sample()  # Replace this with your strategy
    obs, reward, done, info = env.step(action)
    total_reward += reward
    print(f"Action: {action}, Reward: {reward}, Total Reward: {total_reward}")

env.close()

Multi-armed state [0.77395605 0.43887844 0.85859792 0.69736803 0.09417735]
Action: 3, Reward: 0, Total Reward: 0
Action: 3, Reward: 0, Total Reward: 0
Action: 3, Reward: 0, Total Reward: 0
Action: 2, Reward: 1, Total Reward: 1
Action: 2, Reward: 1, Total Reward: 2
Action: 1, Reward: 1, Total Reward: 3
Action: 3, Reward: 0, Total Reward: 3
Action: 4, Reward: 0, Total Reward: 3
Action: 3, Reward: 0, Total Reward: 3
Action: 4, Reward: 0, Total Reward: 3
Action: 1, Reward: 1, Total Reward: 4
Action: 3, Reward: 1, Total Reward: 5
Action: 4, Reward: 1, Total Reward: 6
Action: 2, Reward: 1, Total Reward: 7
Action: 0, Reward: 1, Total Reward: 8
Action: 4, Reward: 0, Total Reward: 8
Action: 1, Reward: 1, Total Reward: 9
Action: 4, Reward: 0, Total Reward: 9
Action: 2, Reward: 0, Total Reward: 9
Action: 1, Reward: 0, Total Reward: 9
Action: 2, Reward: 1, Total Reward: 10
Action: 1, Reward: 0, Total Reward: 10
Action: 2, Reward: 1, Total Reward: 11
Action: 1, Reward: 1, Total Reward: 12
Action: 0